# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902022


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:05<01:23,  2.80s/it]

Rendering models:   9%|▉         | 3/32 [00:07<01:14,  2.58s/it]

Rendering models:  12%|█▎        | 4/32 [00:07<00:52,  1.88s/it]

Rendering models:  16%|█▌        | 5/32 [00:08<00:40,  1.50s/it]

...

Rendering models:  91%|█████████ | 29/32 [00:16<00:01,  2.46it/s]

Rendering models:  94%|█████████▍| 30/32 [00:16<00:00,  2.73it/s]

Rendering models:  97%|█████████▋| 31/32 [00:16<00:00,  3.41it/s]

Rendering models: 100%|██████████| 32/32 [00:17<00:00,  2.67it/s]

equidad1                               0.061547
tingard                                0.000076
bobhail                                0.000100
klmasters                              0.000164
xperry9702                             0.195925
jumpy26                                0.041539
not-logged-in-2f77ea516eada33edf99     0.146500
not-logged-in-20487e5808245554c355     0.000603
OlEnglish                              0.003973
CScampstudent                          0.998944
theetick                               0.003546
sn346808                               0.012132
not-logged-in-76e4e6116d9c15279033     0.045760
not-logged-in-14d09f70263c1fe15cc3     0.000298
LeeJoseph                              0.005674
williamhernandez                       0.000677
ElisabethB                             0.000107
kevinlacaille                          0.016406
manik17                                0.000141
jimaneec                               0.001357
genestf                                0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())